In [447]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np

In [448]:
url = "https://en.wikipedia.org/wiki/List_of_FIFA_World_Cup_penalty_shoot-outs"
r = requests.get(url)
soup = bs(r.content, "lxml")

In [449]:
t = soup.find_all('table')

In [450]:
table = t[1]

In [451]:
rows = table.find_all('tr')

In [452]:
d = {}

In [453]:
d['winners'] = []
for row in rows:
    cells = row.find_all('td')
    if cells:
        d['winners'].append(cells[0].text[1:-1]) #spliting string as plain form is '\xa0West Germany\n'

print(d['winners'])

['West Germany', 'France', 'West Germany', 'Belgium', 'Republic of Ireland', 'Argentina', 'Argentina', 'West Germany', 'Bulgaria', 'Sweden', 'Brazil', 'Argentina', 'France', 'Brazil', 'Spain', 'South Korea', 'Ukraine', 'Germany', 'Portugal', 'Italy', 'Paraguay', 'Uruguay', 'Brazil', 'Costa Rica', 'Netherlands', 'Argentina', 'Russia', 'Croatia', 'England', 'Croatia']


In [454]:
d['losers'] = []
for row in rows:
    cells = row.find_all('td')
    if cells:
        d['losers'].append(cells[2].text[1:-1]) #spliting string as plain form is '\xa0West Germany\n'

print(d['losers'])

['France', 'Brazil', 'Mexico', 'Spain', 'Romania', 'Yugoslavia', 'Italy', 'England', 'Mexico', 'Romania', 'Italy', 'England', 'Italy', 'Netherlands', 'Republic of Ireland', 'Spain', '\xa0Switzerland', 'Argentina', 'England', 'France', 'Japan', 'Ghana', 'Chile', 'Greece', 'Costa Rica', 'Netherlands', 'Spain', 'Denmark', 'Colombia', 'Russia']


In [455]:
def remove_xa0_from_start(startList):
    cleanList = []
    for i in startList:
        if i[0] == '\xa0':
            cleanList.append(i[1:])
        else:
            cleanList.append(i)
    return cleanList

In [456]:
#One value in the losers column still started with \xa0 so making sure they're removed.
d['losers'] = remove_xa0_from_list(d['losers'])
print(d['losers'])
d['winners'] = remove_xa0_from_list(d['winners'])
print(d['winners'])

['France', 'Brazil', 'Mexico', 'Spain', 'Romania', 'Yugoslavia', 'Italy', 'England', 'Mexico', 'Romania', 'Italy', 'England', 'Italy', 'Netherlands', 'Republic of Ireland', 'Spain', 'Switzerland', 'Argentina', 'England', 'France', 'Japan', 'Ghana', 'Chile', 'Greece', 'Costa Rica', 'Netherlands', 'Spain', 'Denmark', 'Colombia', 'Russia']
['West Germany', 'France', 'West Germany', 'Belgium', 'Republic of Ireland', 'Argentina', 'Argentina', 'West Germany', 'Bulgaria', 'Sweden', 'Brazil', 'Argentina', 'France', 'Brazil', 'Spain', 'South Korea', 'Ukraine', 'Germany', 'Portugal', 'Italy', 'Paraguay', 'Uruguay', 'Brazil', 'Costa Rica', 'Netherlands', 'Argentina', 'Russia', 'Croatia', 'England', 'Croatia']


In [457]:
d['game_ID'] = list(range(1, len(d['winners'])+1))

In [458]:
#Games must have ended in a draw to get to penalties so only need one number for goals scored
d['pre_penalties_score'] = []
for row in rows:
    cells = row.find_all('td')
    if cells:
        d['pre_penalties_score'].append(cells[1].text[0])

print(d['pre_penalties_score'])

['3', '1', '0', '1', '0', '0', '1', '1', '1', '2', '0', '2', '0', '1', '1', '0', '0', '1', '0', '1', '0', '1', '1', '1', '0', '0', '1', '1', '1', '2']


In [459]:
print(d)

{'winners': ['West Germany', 'France', 'West Germany', 'Belgium', 'Republic of Ireland', 'Argentina', 'Argentina', 'West Germany', 'Bulgaria', 'Sweden', 'Brazil', 'Argentina', 'France', 'Brazil', 'Spain', 'South Korea', 'Ukraine', 'Germany', 'Portugal', 'Italy', 'Paraguay', 'Uruguay', 'Brazil', 'Costa Rica', 'Netherlands', 'Argentina', 'Russia', 'Croatia', 'England', 'Croatia'], 'losers': ['France', 'Brazil', 'Mexico', 'Spain', 'Romania', 'Yugoslavia', 'Italy', 'England', 'Mexico', 'Romania', 'Italy', 'England', 'Italy', 'Netherlands', 'Republic of Ireland', 'Spain', 'Switzerland', 'Argentina', 'England', 'France', 'Japan', 'Ghana', 'Chile', 'Greece', 'Costa Rica', 'Netherlands', 'Spain', 'Denmark', 'Colombia', 'Russia'], 'game_ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 'pre_penalties_score': ['3', '1', '0', '1', '0', '0', '1', '1', '1', '2', '0', '2', '0', '1', '1', '0', '0', '1', '0', '1', '0', '1', '1', '1', 

In [460]:
def penKickerAndResult(cell, splitToKeep):
    kickersAndResults = []
    for i in range(len(cell[1:])):
        kicker = cell[i+1].split('title="')[splitToKeep].split('"')[0]
        
        #some names showing as "Willian (footballer, born 1988)" so clipping off "(footballer..."
        kicker = kicker.split(' (')[0]
        
        if 'Penalty scored' in cell[i+1]:
            result = True
        else:
            result = False
        
        kickersAndResults.append((kicker, result))
            
    return kickersAndResults

In [461]:
d['winner_penalties'] = []
for i in range(len(rows)):
    cells = rows[i].find_all('td')
    if cells: #need test as the two header rows would cause an error when trying to split
        convertedCell = str(cells[7]).split('<span class="nowrap">')
        d['winner_penalties'].append(penKickerAndResult(convertedCell, 1))

d['winner_penalties']

[[('Manfred Kaltz', True),
  ('Paul Breitner', True),
  ('Uli Stielike', False),
  ('Pierre Littbarski', True),
  ('Karl-Heinz Rummenigge', True),
  ('Horst Hrubesch', True)],
 [('Yannick Stopyra', True),
  ('Manuel Amoros', True),
  ('Bruno Bellone', True),
  ('Michel Platini', False),
  ('Luis Fernández', True)],
 [('Klaus Allofs', True),
  ('Andreas Brehme', True),
  ('Lothar Matthäus', True),
  ('Pierre Littbarski', True)],
 [('Nico Claesen', True),
  ('Enzo Scifo', True),
  ('Hugo Broos', True),
  ('Patrick Vervoort', True),
  ('Leo Van der Elst', True)],
 [('Kevin Sheedy', True),
  ('Ray Houghton', True),
  ('Andy Townsend', True),
  ('Tony Cascarino', True),
  ("David O'Leary", True)],
 [('José Serrizuela', True),
  ('Jorge Burruchaga', True),
  ('Diego Maradona', False),
  ('Pedro Troglio', False),
  ('Gustavo Dezotti', True)],
 [('José Serrizuela', True),
  ('Jorge Burruchaga', True),
  ('Julio Olarticoechea', True),
  ('Diego Maradona', True)],
 [('Andreas Brehme', True),
  (

In [462]:
d['loser_penalties'] = []
for i in range(len(rows)):
    cells = rows[i].find_all('td')
    if cells: #need test as the two header rows would cause an error when trying to split
        convertedCell = str(cells[8]).split('<span class="nowrap">')
        d['loser_penalties'].append(penKickerAndResult(convertedCell, 2))

d['loser_penalties']

[[('Alain Giresse', True),
  ('Manuel Amoros', True),
  ('Dominique Rocheteau', True),
  ('Didier Six', False),
  ('Michel Platini', True),
  ('Maxime Bossis', False)],
 [('Sócrates', False),
  ('Alemão', True),
  ('Zico', True),
  ('Branco', True),
  ('Júlio César', False)],
 [('Manuel Negrete Arias', True),
  ('Fernando Quirarte', False),
  ('Raúl Servín', False)],
 [('Juan Antonio Señor', True),
  ('Eloy Olaya', False),
  ('Chendo', True),
  ('Emilio Butragueño', True),
  ('Víctor Muñoz', True)],
 [('Gheorghe Hagi', True),
  ('Dănuț Lupu', True),
  ('Iosif Rotariu', True),
  ('Ioan Lupescu', True),
  ('Daniel Timofte', False)],
 [('Dragan Stojković', False),
  ('Robert Prosinečki', True),
  ('Dejan Savićević', True),
  ('Dragoljub Brnović', False),
  ('Faruk Hadžibegić', False)],
 [('Franco Baresi', True),
  ('Roberto Baggio', True),
  ('Luigi De Agostini', True),
  ('Roberto Donadoni', False),
  ('Aldo Serena', False)],
 [('Gary Lineker', True),
  ('Peter Beardsley', True),
  ('Dav

In [463]:
#if team won it's first player will be highlighted with a grey background so test if 
#that color's tag is in the cell

d['winner_took_first_penalty'] = []
for i in range(len(rows)):
    cells = rows[i].find_all('td')
    if cells: #need test as the two header rows would cause an error when trying to split
        if 'background-color: #C3C3C3' in str(cells[7]):
            d['winner_took_first_penalty'].append(True)
        else:
            d['winner_took_first_penalty'].append(False)
        
print(d['winner_took_first_penalty'])

[False, False, True, False, False, True, False, False, False, True, False, True, True, True, False, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False]


In [464]:
d['year'] = []
d['host_country'] = []

i = 2 #skip rows 0 and 1 as they're headers
loopAgain = True
while loopAgain:
    cell = rows[i].find_all('td')[10]
    yearAndLocationSplit = cell.text[:-1].split(', ') #[:-1] to strip '\n' off end
    
    d['year'].append(int(yearAndLocationSplit[0]))
    d['host_country'].append(yearAndLocationSplit[1])
    
    i += 1
    
    #deal with cells that span multiple rows.
    if "rowspan" in str(cell):
        extraloops = int(str(cell).split('"')[1]) - 1 #-1 as already done one loop above
        for j in range(extraloops):
            d['year'].append(int(yearAndLocationSplit[0]))
            d['host_country'].append(yearAndLocationSplit[1])
            i += 1
    
    if i >= len(rows):
        loopAgain = False
        
print(d['year'])
print(d['host_country'])

[1982, 1986, 1986, 1986, 1990, 1990, 1990, 1990, 1994, 1994, 1994, 1998, 1998, 1998, 2002, 2002, 2006, 2006, 2006, 2006, 2010, 2010, 2014, 2014, 2014, 2014, 2018, 2018, 2018, 2018]
['Spain', 'Mexico', 'Mexico', 'Mexico', 'Italy', 'Italy', 'Italy', 'Italy', 'USA', 'USA', 'USA', 'France', 'France', 'France', 'Korea/Japan', 'Korea/Japan', 'Germany', 'Germany', 'Germany', 'Germany', 'South Africa', 'South Africa', 'Brazil', 'Brazil', 'Brazil', 'Brazil', 'Russia', 'Russia', 'Russia', 'Russia']


In [465]:
d['stage'] = []

i = 2 #skip rows 0 and 1 as they're headers
loopAgain = True
while loopAgain:
    previousColCell = rows[i].find_all('td')[10]
    
    if '#' in str(previousColCell):
        cell = previousColCell
        stageVar = previousColCell.text
    elif ('style="display:none;"' in str(previousColCell) or 
          'style="white-space:nowrap"' in str(previousColCell)):
        #work around for dealing with game8 where there's a merged
        #cell for stage, but it doesn't start on the same row as the 
        #merged cell for the year/location.
        
        #leave cell as value from last loop
        pass
    else:
        cell = rows[i].find_all('td')[11]
    
    stageVar = cell.text[1:-1] #[1:-1] as text comes in as ' Semi-finals\n'
    d['stage'].append(stageVar)
    
    i += 1
    
    #deal with cells that define how many rows they span
    if "rowspan" in str(cell):
        extraloops = int(str(cell).split('"')[1]) - 1 #-1 as already done one loop above
        for j in range(extraloops):
            d['stage'].append(stageVar)
            i += 1
    
    if i >= len(rows):
        loopAgain = False
        
print(d['stage'])

['Semi-finals', 'Quarter-finals', 'Quarter-finals', 'Quarter-finals', 'Second round', 'Quarter-finals', 'Quarter-finals', 'Quarter-finals', 'Second round', 'Quarter-finals', 'Final', 'Second round', 'Quarter-finals', 'Semi-finals', 'Second round', 'Quarter-finals', 'Second round', 'Quarter-finals', 'Quarter-finals', 'Final', 'Second round', 'Quarter-finals', 'Second round', 'Second round', 'Quarter-finals', 'Semi-finals', 'Second round', 'Second round', 'Second round', 'Quarter-finals']


In [466]:
df = pd.DataFrame(d)
df

,game_ID,host_country,loser_penalties,losers,pre_penalties_score,stage,winner_penalties,winner_took_first_penalty,winners,year
0,1,Spain,"[(Alain Giresse, True), (Manuel Amoros, True),...",France,3,Semi-finals,"[(Manfred Kaltz, True), (Paul Breitner, True),...",False,West Germany,1982
1,2,Mexico,"[(Sócrates, False), (Alemão, True), (Zico, Tru...",Brazil,1,Quarter-finals,"[(Yannick Stopyra, True), (Manuel Amoros, True...",False,France,1986
2,3,Mexico,"[(Manuel Negrete Arias, True), (Fernando Quira...",Mexico,0,Quarter-finals,"[(Klaus Allofs, True), (Andreas Brehme, True),...",True,West Germany,1986
3,4,Mexico,"[(Juan Antonio Señor, True), (Eloy Olaya, Fals...",Spain,1,Quarter-finals,"[(Nico Claesen, True), (Enzo Scifo, True), (Hu...",False,Belgium,1986
4,5,Italy,"[(Gheorghe Hagi, True), (Dănuț Lupu, True), (I...",Romania,0,Second round,"[(Kevin Sheedy, True), (Ray Houghton, True), (...",False,Republic of Ireland,1990
5,6,Italy,"[(Dragan Stojković, False), (Robert Prosinečki...",Yugoslavia,0,Quarter-finals,"[(José Serrizuela, True), (Jorge Burruchaga, T...",True,Argentina,1990
6,7,Italy,"[(Franco Baresi, True), (Roberto Baggio, True)...",Italy,1,Quarter-finals,"[(José Serrizuela, True), (Jorge Burruchaga, T...",False,Argentina,1990
7,8,Italy,"[(Gary Lineker, True), (Peter Beardsley, True)...",England,1,Quarter-finals,"[(Andreas Brehme, True), (Lothar Matthäus, Tru...",False,West Germany,1990
8,9,USA,"[(Alberto García Aspe, False), (Marcelino Bern...",Mexico,1,Second round,"[(Krasimir Balakov, False), (Boncho Genchev, T...",False,Bulgaria,1994
9,10,USA,"[(Florin Răducioiu, True), (Gheorghe Hagi, Tru...",Romania,2,Quarter-finals,"[(Håkan Mild, False), (Kennet Andersson, True)...",True,Sweden,1994


In [467]:
df.columns

Index(['game_ID', 'host_country', 'loser_penalties', 'losers',
       'pre_penalties_score', 'stage', 'winner_penalties',
       'winner_took_first_penalty', 'winners', 'year'],
      dtype='object')

In [468]:
nonWinLoserCols = list(df.columns)
nonWinLoserCols

['game_ID',
 'host_country',
 'loser_penalties',
 'losers',
 'pre_penalties_score',
 'stage',
 'winner_penalties',
 'winner_took_first_penalty',
 'winners',
 'year']

In [469]:
nonWinLoserCols.remove('winners')
nonWinLoserCols.remove('losers')
nonWinLoserCols

['game_ID',
 'host_country',
 'loser_penalties',
 'pre_penalties_score',
 'stage',
 'winner_penalties',
 'winner_took_first_penalty',
 'year']

In [470]:
df = df.melt(id_vars = nonWinLoserCols, value_vars = ['winners', 'losers'], 
             var_name='won', value_name='team')
df

,game_ID,host_country,loser_penalties,pre_penalties_score,stage,winner_penalties,winner_took_first_penalty,year,won,team
0,1,Spain,"[(Alain Giresse, True), (Manuel Amoros, True),...",3,Semi-finals,"[(Manfred Kaltz, True), (Paul Breitner, True),...",False,1982,winners,West Germany
1,2,Mexico,"[(Sócrates, False), (Alemão, True), (Zico, Tru...",1,Quarter-finals,"[(Yannick Stopyra, True), (Manuel Amoros, True...",False,1986,winners,France
2,3,Mexico,"[(Manuel Negrete Arias, True), (Fernando Quira...",0,Quarter-finals,"[(Klaus Allofs, True), (Andreas Brehme, True),...",True,1986,winners,West Germany
3,4,Mexico,"[(Juan Antonio Señor, True), (Eloy Olaya, Fals...",1,Quarter-finals,"[(Nico Claesen, True), (Enzo Scifo, True), (Hu...",False,1986,winners,Belgium
4,5,Italy,"[(Gheorghe Hagi, True), (Dănuț Lupu, True), (I...",0,Second round,"[(Kevin Sheedy, True), (Ray Houghton, True), (...",False,1990,winners,Republic of Ireland
5,6,Italy,"[(Dragan Stojković, False), (Robert Prosinečki...",0,Quarter-finals,"[(José Serrizuela, True), (Jorge Burruchaga, T...",True,1990,winners,Argentina
6,7,Italy,"[(Franco Baresi, True), (Roberto Baggio, True)...",1,Quarter-finals,"[(José Serrizuela, True), (Jorge Burruchaga, T...",False,1990,winners,Argentina
7,8,Italy,"[(Gary Lineker, True), (Peter Beardsley, True)...",1,Quarter-finals,"[(Andreas Brehme, True), (Lothar Matthäus, Tru...",False,1990,winners,West Germany
8,9,USA,"[(Alberto García Aspe, False), (Marcelino Bern...",1,Second round,"[(Krasimir Balakov, False), (Boncho Genchev, T...",False,1994,winners,Bulgaria
9,10,USA,"[(Florin Răducioiu, True), (Gheorghe Hagi, Tru...",2,Quarter-finals,"[(Håkan Mild, False), (Kennet Andersson, True)...",True,1994,winners,Sweden


In [471]:
df.won = df.won.apply(lambda x: True if x == 'winners' else False)
df

,game_ID,host_country,loser_penalties,pre_penalties_score,stage,winner_penalties,winner_took_first_penalty,year,won,team
0,1,Spain,"[(Alain Giresse, True), (Manuel Amoros, True),...",3,Semi-finals,"[(Manfred Kaltz, True), (Paul Breitner, True),...",False,1982,True,West Germany
1,2,Mexico,"[(Sócrates, False), (Alemão, True), (Zico, Tru...",1,Quarter-finals,"[(Yannick Stopyra, True), (Manuel Amoros, True...",False,1986,True,France
2,3,Mexico,"[(Manuel Negrete Arias, True), (Fernando Quira...",0,Quarter-finals,"[(Klaus Allofs, True), (Andreas Brehme, True),...",True,1986,True,West Germany
3,4,Mexico,"[(Juan Antonio Señor, True), (Eloy Olaya, Fals...",1,Quarter-finals,"[(Nico Claesen, True), (Enzo Scifo, True), (Hu...",False,1986,True,Belgium
4,5,Italy,"[(Gheorghe Hagi, True), (Dănuț Lupu, True), (I...",0,Second round,"[(Kevin Sheedy, True), (Ray Houghton, True), (...",False,1990,True,Republic of Ireland
5,6,Italy,"[(Dragan Stojković, False), (Robert Prosinečki...",0,Quarter-finals,"[(José Serrizuela, True), (Jorge Burruchaga, T...",True,1990,True,Argentina
6,7,Italy,"[(Franco Baresi, True), (Roberto Baggio, True)...",1,Quarter-finals,"[(José Serrizuela, True), (Jorge Burruchaga, T...",False,1990,True,Argentina
7,8,Italy,"[(Gary Lineker, True), (Peter Beardsley, True)...",1,Quarter-finals,"[(Andreas Brehme, True), (Lothar Matthäus, Tru...",False,1990,True,West Germany
8,9,USA,"[(Alberto García Aspe, False), (Marcelino Bern...",1,Second round,"[(Krasimir Balakov, False), (Boncho Genchev, T...",False,1994,True,Bulgaria
9,10,USA,"[(Florin Răducioiu, True), (Gheorghe Hagi, Tru...",2,Quarter-finals,"[(Håkan Mild, False), (Kennet Andersson, True)...",True,1994,True,Sweden


In [472]:
df['pen_kicks_and_results'] = np.nan
df

,game_ID,host_country,loser_penalties,pre_penalties_score,stage,winner_penalties,winner_took_first_penalty,year,won,team,pen_kicks_and_results
0,1,Spain,"[(Alain Giresse, True), (Manuel Amoros, True),...",3,Semi-finals,"[(Manfred Kaltz, True), (Paul Breitner, True),...",False,1982,True,West Germany,NaN
1,2,Mexico,"[(Sócrates, False), (Alemão, True), (Zico, Tru...",1,Quarter-finals,"[(Yannick Stopyra, True), (Manuel Amoros, True...",False,1986,True,France,NaN
2,3,Mexico,"[(Manuel Negrete Arias, True), (Fernando Quira...",0,Quarter-finals,"[(Klaus Allofs, True), (Andreas Brehme, True),...",True,1986,True,West Germany,NaN
3,4,Mexico,"[(Juan Antonio Señor, True), (Eloy Olaya, Fals...",1,Quarter-finals,"[(Nico Claesen, True), (Enzo Scifo, True), (Hu...",False,1986,True,Belgium,NaN
4,5,Italy,"[(Gheorghe Hagi, True), (Dănuț Lupu, True), (I...",0,Second round,"[(Kevin Sheedy, True), (Ray Houghton, True), (...",False,1990,True,Republic of Ireland,NaN
5,6,Italy,"[(Dragan Stojković, False), (Robert Prosinečki...",0,Quarter-finals,"[(José Serrizuela, True), (Jorge Burruchaga, T...",True,1990,True,Argentina,NaN
6,7,Italy,"[(Franco Baresi, True), (Roberto Baggio, True)...",1,Quarter-finals,"[(José Serrizuela, True), (Jorge Burruchaga, T...",False,1990,True,Argentina,NaN
7,8,Italy,"[(Gary Lineker, True), (Peter Beardsley, True)...",1,Quarter-finals,"[(Andreas Brehme, True), (Lothar Matthäus, Tru...",False,1990,True,West Germany,NaN
8,9,USA,"[(Alberto García Aspe, False), (Marcelino Bern...",1,Second round,"[(Krasimir Balakov, False), (Boncho Genchev, T...",False,1994,True,Bulgaria,NaN
9,10,USA,"[(Florin Răducioiu, True), (Gheorghe Hagi, Tru...",2,Quarter-finals,"[(Håkan Mild, False), (Kennet Andersson, True)...",True,1994,True,Sweden,NaN


In [473]:
def rowPenaltyDetails(row):
    if row.won:
        return row.winner_penalties
    else:
        return row.loser_penalties    

In [474]:
df.pen_kicks_and_results = df.apply(rowPenaltyDetails, axis = 1)
df

,game_ID,host_country,loser_penalties,pre_penalties_score,stage,winner_penalties,winner_took_first_penalty,year,won,team,pen_kicks_and_results
0,1,Spain,"[(Alain Giresse, True), (Manuel Amoros, True),...",3,Semi-finals,"[(Manfred Kaltz, True), (Paul Breitner, True),...",False,1982,True,West Germany,"[(Manfred Kaltz, True), (Paul Breitner, True),..."
1,2,Mexico,"[(Sócrates, False), (Alemão, True), (Zico, Tru...",1,Quarter-finals,"[(Yannick Stopyra, True), (Manuel Amoros, True...",False,1986,True,France,"[(Yannick Stopyra, True), (Manuel Amoros, True..."
2,3,Mexico,"[(Manuel Negrete Arias, True), (Fernando Quira...",0,Quarter-finals,"[(Klaus Allofs, True), (Andreas Brehme, True),...",True,1986,True,West Germany,"[(Klaus Allofs, True), (Andreas Brehme, True),..."
3,4,Mexico,"[(Juan Antonio Señor, True), (Eloy Olaya, Fals...",1,Quarter-finals,"[(Nico Claesen, True), (Enzo Scifo, True), (Hu...",False,1986,True,Belgium,"[(Nico Claesen, True), (Enzo Scifo, True), (Hu..."
4,5,Italy,"[(Gheorghe Hagi, True), (Dănuț Lupu, True), (I...",0,Second round,"[(Kevin Sheedy, True), (Ray Houghton, True), (...",False,1990,True,Republic of Ireland,"[(Kevin Sheedy, True), (Ray Houghton, True), (..."
5,6,Italy,"[(Dragan Stojković, False), (Robert Prosinečki...",0,Quarter-finals,"[(José Serrizuela, True), (Jorge Burruchaga, T...",True,1990,True,Argentina,"[(José Serrizuela, True), (Jorge Burruchaga, T..."
6,7,Italy,"[(Franco Baresi, True), (Roberto Baggio, True)...",1,Quarter-finals,"[(José Serrizuela, True), (Jorge Burruchaga, T...",False,1990,True,Argentina,"[(José Serrizuela, True), (Jorge Burruchaga, T..."
7,8,Italy,"[(Gary Lineker, True), (Peter Beardsley, True)...",1,Quarter-finals,"[(Andreas Brehme, True), (Lothar Matthäus, Tru...",False,1990,True,West Germany,"[(Andreas Brehme, True), (Lothar Matthäus, Tru..."
8,9,USA,"[(Alberto García Aspe, False), (Marcelino Bern...",1,Second round,"[(Krasimir Balakov, False), (Boncho Genchev, T...",False,1994,True,Bulgaria,"[(Krasimir Balakov, False), (Boncho Genchev, T..."
9,10,USA,"[(Florin Răducioiu, True), (Gheorghe Hagi, Tru...",2,Quarter-finals,"[(Håkan Mild, False), (Kennet Andersson, True)...",True,1994,True,Sweden,"[(Håkan Mild, False), (Kennet Andersson, True)..."


In [475]:
df.drop(['winner_penalties', 'loser_penalties'], axis = 1, inplace = True)

In [476]:
df

,game_ID,host_country,pre_penalties_score,stage,winner_took_first_penalty,year,won,team,pen_kicks_and_results
0,1,Spain,3,Semi-finals,False,1982,True,West Germany,"[(Manfred Kaltz, True), (Paul Breitner, True),..."
1,2,Mexico,1,Quarter-finals,False,1986,True,France,"[(Yannick Stopyra, True), (Manuel Amoros, True..."
2,3,Mexico,0,Quarter-finals,True,1986,True,West Germany,"[(Klaus Allofs, True), (Andreas Brehme, True),..."
3,4,Mexico,1,Quarter-finals,False,1986,True,Belgium,"[(Nico Claesen, True), (Enzo Scifo, True), (Hu..."
4,5,Italy,0,Second round,False,1990,True,Republic of Ireland,"[(Kevin Sheedy, True), (Ray Houghton, True), (..."
5,6,Italy,0,Quarter-finals,True,1990,True,Argentina,"[(José Serrizuela, True), (Jorge Burruchaga, T..."
6,7,Italy,1,Quarter-finals,False,1990,True,Argentina,"[(José Serrizuela, True), (Jorge Burruchaga, T..."
7,8,Italy,1,Quarter-finals,False,1990,True,West Germany,"[(Andreas Brehme, True), (Lothar Matthäus, Tru..."
8,9,USA,1,Second round,False,1994,True,Bulgaria,"[(Krasimir Balakov, False), (Boncho Genchev, T..."
9,10,USA,2,Quarter-finals,True,1994,True,Sweden,"[(Håkan Mild, False), (Kennet Andersson, True)..."


In [477]:
#create columns to seperate the penalties kicker/result out into
#by finding the max amount of rounds we can make enough columns for all rows
#afterwards table will be melted and the nan rows of shorter games can be deleted

maxRoundsOfPenalties = max(df.pen_kicks_and_results.apply(len))
maxRoundsOfPenalties

6

In [478]:
for i in range(maxRoundsOfPenalties):
    df[i+1] = np.nan

In [479]:
df

,game_ID,host_country,pre_penalties_score,stage,winner_took_first_penalty,year,won,team,pen_kicks_and_results,1,2,3,4,5,6
0,1,Spain,3,Semi-finals,False,1982,True,West Germany,"[(Manfred Kaltz, True), (Paul Breitner, True),...",NaN,NaN,NaN,NaN,NaN,NaN
1,2,Mexico,1,Quarter-finals,False,1986,True,France,"[(Yannick Stopyra, True), (Manuel Amoros, True...",NaN,NaN,NaN,NaN,NaN,NaN
2,3,Mexico,0,Quarter-finals,True,1986,True,West Germany,"[(Klaus Allofs, True), (Andreas Brehme, True),...",NaN,NaN,NaN,NaN,NaN,NaN
3,4,Mexico,1,Quarter-finals,False,1986,True,Belgium,"[(Nico Claesen, True), (Enzo Scifo, True), (Hu...",NaN,NaN,NaN,NaN,NaN,NaN
4,5,Italy,0,Second round,False,1990,True,Republic of Ireland,"[(Kevin Sheedy, True), (Ray Houghton, True), (...",NaN,NaN,NaN,NaN,NaN,NaN
5,6,Italy,0,Quarter-finals,True,1990,True,Argentina,"[(José Serrizuela, True), (Jorge Burruchaga, T...",NaN,NaN,NaN,NaN,NaN,NaN
6,7,Italy,1,Quarter-finals,False,1990,True,Argentina,"[(José Serrizuela, True), (Jorge Burruchaga, T...",NaN,NaN,NaN,NaN,NaN,NaN
7,8,Italy,1,Quarter-finals,False,1990,True,West Germany,"[(Andreas Brehme, True), (Lothar Matthäus, Tru...",NaN,NaN,NaN,NaN,NaN,NaN
8,9,USA,1,Second round,False,1994,True,Bulgaria,"[(Krasimir Balakov, False), (Boncho Genchev, T...",NaN,NaN,NaN,NaN,NaN,NaN
9,10,USA,2,Quarter-finals,True,1994,True,Sweden,"[(Håkan Mild, False), (Kennet Andersson, True)...",NaN,NaN,NaN,NaN,NaN,NaN


In [480]:
def splitOutPenKickersAndResults(row):
    vals = [np.nan]*maxRoundsOfPenalties
    for i in range(len(row.pen_kicks_and_results)):
        vals[i] = row.pen_kicks_and_results[i]
    return pd.Series(vals) #casting as series as throws error about 'Must have equal len keys and value...' if it's not

In [481]:
penColsList = list(range(1, maxRoundsOfPenalties + 1))
penColsList

[1, 2, 3, 4, 5, 6]

In [482]:
df[penColsList] = df.apply(splitOutPenKickersAndResults, axis=1)

In [483]:
df

,game_ID,host_country,pre_penalties_score,stage,winner_took_first_penalty,year,won,team,pen_kicks_and_results,1,2,3,4,5,6
0,1,Spain,3,Semi-finals,False,1982,True,West Germany,"[(Manfred Kaltz, True), (Paul Breitner, True),...","(Manfred Kaltz, True)","(Paul Breitner, True)","(Uli Stielike, False)","(Pierre Littbarski, True)","(Karl-Heinz Rummenigge, True)","(Horst Hrubesch, True)"
1,2,Mexico,1,Quarter-finals,False,1986,True,France,"[(Yannick Stopyra, True), (Manuel Amoros, True...","(Yannick Stopyra, True)","(Manuel Amoros, True)","(Bruno Bellone, True)","(Michel Platini, False)","(Luis Fernández, True)",NaN
2,3,Mexico,0,Quarter-finals,True,1986,True,West Germany,"[(Klaus Allofs, True), (Andreas Brehme, True),...","(Klaus Allofs, True)","(Andreas Brehme, True)","(Lothar Matthäus, True)","(Pierre Littbarski, True)",NaN,NaN
3,4,Mexico,1,Quarter-finals,False,1986,True,Belgium,"[(Nico Claesen, True), (Enzo Scifo, True), (Hu...","(Nico Claesen, True)","(Enzo Scifo, True)","(Hugo Broos, True)","(Patrick Vervoort, True)","(Leo Van der Elst, True)",NaN
4,5,Italy,0,Second round,False,1990,True,Republic of Ireland,"[(Kevin Sheedy, True), (Ray Houghton, True), (...","(Kevin Sheedy, True)","(Ray Houghton, True)","(Andy Townsend, True)","(Tony Cascarino, True)","(David O'Leary, True)",NaN
5,6,Italy,0,Quarter-finals,True,1990,True,Argentina,"[(José Serrizuela, True), (Jorge Burruchaga, T...","(José Serrizuela, True)","(Jorge Burruchaga, True)","(Diego Maradona, False)","(Pedro Troglio, False)","(Gustavo Dezotti, True)",NaN
6,7,Italy,1,Quarter-finals,False,1990,True,Argentina,"[(José Serrizuela, True), (Jorge Burruchaga, T...","(José Serrizuela, True)","(Jorge Burruchaga, True)","(Julio Olarticoechea, True)","(Diego Maradona, True)",NaN,NaN
7,8,Italy,1,Quarter-finals,False,1990,True,West Germany,"[(Andreas Brehme, True), (Lothar Matthäus, Tru...","(Andreas Brehme, True)","(Lothar Matthäus, True)","(Karl-Heinz Riedle, True)","(Olaf Thon, True)",NaN,NaN
8,9,USA,1,Second round,False,1994,True,Bulgaria,"[(Krasimir Balakov, False), (Boncho Genchev, T...","(Krasimir Balakov, False)","(Boncho Genchev, True)","(Daniel Borimirov, True)","(Yordan Letchkov, True)",NaN,NaN
9,10,USA,2,Quarter-finals,True,1994,True,Sweden,"[(Håkan Mild, False), (Kennet Andersson, True)...","(Håkan Mild, False)","(Kennet Andersson, True)","(Tomas Brolin, True)","(Klas Ingesson, True)","(Roland Nilsson, True)","(Henrik Larsson, True)"


In [484]:
df.drop('pen_kicks_and_results', axis = 1, inplace = True)
df

,game_ID,host_country,pre_penalties_score,stage,winner_took_first_penalty,year,won,team,1,2,3,4,5,6
0,1,Spain,3,Semi-finals,False,1982,True,West Germany,"(Manfred Kaltz, True)","(Paul Breitner, True)","(Uli Stielike, False)","(Pierre Littbarski, True)","(Karl-Heinz Rummenigge, True)","(Horst Hrubesch, True)"
1,2,Mexico,1,Quarter-finals,False,1986,True,France,"(Yannick Stopyra, True)","(Manuel Amoros, True)","(Bruno Bellone, True)","(Michel Platini, False)","(Luis Fernández, True)",NaN
2,3,Mexico,0,Quarter-finals,True,1986,True,West Germany,"(Klaus Allofs, True)","(Andreas Brehme, True)","(Lothar Matthäus, True)","(Pierre Littbarski, True)",NaN,NaN
3,4,Mexico,1,Quarter-finals,False,1986,True,Belgium,"(Nico Claesen, True)","(Enzo Scifo, True)","(Hugo Broos, True)","(Patrick Vervoort, True)","(Leo Van der Elst, True)",NaN
4,5,Italy,0,Second round,False,1990,True,Republic of Ireland,"(Kevin Sheedy, True)","(Ray Houghton, True)","(Andy Townsend, True)","(Tony Cascarino, True)","(David O'Leary, True)",NaN
5,6,Italy,0,Quarter-finals,True,1990,True,Argentina,"(José Serrizuela, True)","(Jorge Burruchaga, True)","(Diego Maradona, False)","(Pedro Troglio, False)","(Gustavo Dezotti, True)",NaN
6,7,Italy,1,Quarter-finals,False,1990,True,Argentina,"(José Serrizuela, True)","(Jorge Burruchaga, True)","(Julio Olarticoechea, True)","(Diego Maradona, True)",NaN,NaN
7,8,Italy,1,Quarter-finals,False,1990,True,West Germany,"(Andreas Brehme, True)","(Lothar Matthäus, True)","(Karl-Heinz Riedle, True)","(Olaf Thon, True)",NaN,NaN
8,9,USA,1,Second round,False,1994,True,Bulgaria,"(Krasimir Balakov, False)","(Boncho Genchev, True)","(Daniel Borimirov, True)","(Yordan Letchkov, True)",NaN,NaN
9,10,USA,2,Quarter-finals,True,1994,True,Sweden,"(Håkan Mild, False)","(Kennet Andersson, True)","(Tomas Brolin, True)","(Klas Ingesson, True)","(Roland Nilsson, True)","(Henrik Larsson, True)"


In [485]:
nonPenCols = set(list(df.columns)) - set(penColsList)

In [486]:
df = df.melt(id_vars = nonPenCols, value_vars = penColsList, 
        var_name='penalty_round', value_name='kicker_and_result')

In [487]:
df

,stage,pre_penalties_score,winner_took_first_penalty,team,year,host_country,game_ID,won,penalty_round,kicker_and_result
0,Semi-finals,3,False,West Germany,1982,Spain,1,True,1,"(Manfred Kaltz, True)"
1,Quarter-finals,1,False,France,1986,Mexico,2,True,1,"(Yannick Stopyra, True)"
2,Quarter-finals,0,True,West Germany,1986,Mexico,3,True,1,"(Klaus Allofs, True)"
3,Quarter-finals,1,False,Belgium,1986,Mexico,4,True,1,"(Nico Claesen, True)"
4,Second round,0,False,Republic of Ireland,1990,Italy,5,True,1,"(Kevin Sheedy, True)"
5,Quarter-finals,0,True,Argentina,1990,Italy,6,True,1,"(José Serrizuela, True)"
6,Quarter-finals,1,False,Argentina,1990,Italy,7,True,1,"(José Serrizuela, True)"
7,Quarter-finals,1,False,West Germany,1990,Italy,8,True,1,"(Andreas Brehme, True)"
8,Second round,1,False,Bulgaria,1994,USA,9,True,1,"(Krasimir Balakov, False)"
9,Quarter-finals,2,True,Sweden,1994,USA,10,True,1,"(Håkan Mild, False)"


In [488]:
df.dropna(subset=['kicker_and_result'], inplace=True)
df

,stage,pre_penalties_score,winner_took_first_penalty,team,year,host_country,game_ID,won,penalty_round,kicker_and_result
0,Semi-finals,3,False,West Germany,1982,Spain,1,True,1,"(Manfred Kaltz, True)"
1,Quarter-finals,1,False,France,1986,Mexico,2,True,1,"(Yannick Stopyra, True)"
2,Quarter-finals,0,True,West Germany,1986,Mexico,3,True,1,"(Klaus Allofs, True)"
3,Quarter-finals,1,False,Belgium,1986,Mexico,4,True,1,"(Nico Claesen, True)"
4,Second round,0,False,Republic of Ireland,1990,Italy,5,True,1,"(Kevin Sheedy, True)"
5,Quarter-finals,0,True,Argentina,1990,Italy,6,True,1,"(José Serrizuela, True)"
6,Quarter-finals,1,False,Argentina,1990,Italy,7,True,1,"(José Serrizuela, True)"
7,Quarter-finals,1,False,West Germany,1990,Italy,8,True,1,"(Andreas Brehme, True)"
8,Second round,1,False,Bulgaria,1994,USA,9,True,1,"(Krasimir Balakov, False)"
9,Quarter-finals,2,True,Sweden,1994,USA,10,True,1,"(Håkan Mild, False)"


In [489]:
df['penalty_kicker'] = df.apply(lambda x: x.kicker_and_result[0], axis=1)
df

,stage,pre_penalties_score,winner_took_first_penalty,team,year,host_country,game_ID,won,penalty_round,kicker_and_result,penalty_kicker
0,Semi-finals,3,False,West Germany,1982,Spain,1,True,1,"(Manfred Kaltz, True)",Manfred Kaltz
1,Quarter-finals,1,False,France,1986,Mexico,2,True,1,"(Yannick Stopyra, True)",Yannick Stopyra
2,Quarter-finals,0,True,West Germany,1986,Mexico,3,True,1,"(Klaus Allofs, True)",Klaus Allofs
3,Quarter-finals,1,False,Belgium,1986,Mexico,4,True,1,"(Nico Claesen, True)",Nico Claesen
4,Second round,0,False,Republic of Ireland,1990,Italy,5,True,1,"(Kevin Sheedy, True)",Kevin Sheedy
5,Quarter-finals,0,True,Argentina,1990,Italy,6,True,1,"(José Serrizuela, True)",José Serrizuela
6,Quarter-finals,1,False,Argentina,1990,Italy,7,True,1,"(José Serrizuela, True)",José Serrizuela
7,Quarter-finals,1,False,West Germany,1990,Italy,8,True,1,"(Andreas Brehme, True)",Andreas Brehme
8,Second round,1,False,Bulgaria,1994,USA,9,True,1,"(Krasimir Balakov, False)",Krasimir Balakov
9,Quarter-finals,2,True,Sweden,1994,USA,10,True,1,"(Håkan Mild, False)",Håkan Mild


In [490]:
df['penalty_scored'] = df.apply(lambda x: x.kicker_and_result[1], axis=1)
df

,stage,pre_penalties_score,winner_took_first_penalty,team,year,host_country,game_ID,won,penalty_round,kicker_and_result,penalty_kicker,penalty_scored
0,Semi-finals,3,False,West Germany,1982,Spain,1,True,1,"(Manfred Kaltz, True)",Manfred Kaltz,True
1,Quarter-finals,1,False,France,1986,Mexico,2,True,1,"(Yannick Stopyra, True)",Yannick Stopyra,True
2,Quarter-finals,0,True,West Germany,1986,Mexico,3,True,1,"(Klaus Allofs, True)",Klaus Allofs,True
3,Quarter-finals,1,False,Belgium,1986,Mexico,4,True,1,"(Nico Claesen, True)",Nico Claesen,True
4,Second round,0,False,Republic of Ireland,1990,Italy,5,True,1,"(Kevin Sheedy, True)",Kevin Sheedy,True
5,Quarter-finals,0,True,Argentina,1990,Italy,6,True,1,"(José Serrizuela, True)",José Serrizuela,True
6,Quarter-finals,1,False,Argentina,1990,Italy,7,True,1,"(José Serrizuela, True)",José Serrizuela,True
7,Quarter-finals,1,False,West Germany,1990,Italy,8,True,1,"(Andreas Brehme, True)",Andreas Brehme,True
8,Second round,1,False,Bulgaria,1994,USA,9,True,1,"(Krasimir Balakov, False)",Krasimir Balakov,False
9,Quarter-finals,2,True,Sweden,1994,USA,10,True,1,"(Håkan Mild, False)",Håkan Mild,False


In [491]:
df.drop('kicker_and_result', axis = 1, inplace = True)
df.head()

,stage,pre_penalties_score,winner_took_first_penalty,team,year,host_country,game_ID,won,penalty_round,penalty_kicker,penalty_scored
0,Semi-finals,3,False,West Germany,1982,Spain,1,True,1,Manfred Kaltz,True
1,Quarter-finals,1,False,France,1986,Mexico,2,True,1,Yannick Stopyra,True
2,Quarter-finals,0,True,West Germany,1986,Mexico,3,True,1,Klaus Allofs,True
3,Quarter-finals,1,False,Belgium,1986,Mexico,4,True,1,Nico Claesen,True
4,Second round,0,False,Republic of Ireland,1990,Italy,5,True,1,Kevin Sheedy,True


In [492]:
df.columns.tolist()

['stage',
 'pre_penalties_score',
 'winner_took_first_penalty',
 'team',
 'year',
 'host_country',
 'game_ID',
 'won',
 'penalty_round',
 'penalty_kicker',
 'penalty_scored']

In [493]:
newColOrder = ['game_ID',
               'year',
               'host_country', 
               'stage',
               'team',
               'pre_penalties_score', 
               'won', 
               'winner_took_first_penalty',
               'penalty_round',
               'penalty_scored']

In [495]:
df = df[newColOrder]
df.head()

,game_ID,year,host_country,stage,team,pre_penalties_score,won,winner_took_first_penalty,penalty_round,penalty_scored
0,1,1982,Spain,Semi-finals,West Germany,3,True,False,1,True
1,2,1986,Mexico,Quarter-finals,France,1,True,False,1,True
2,3,1986,Mexico,Quarter-finals,West Germany,0,True,True,1,True
3,4,1986,Mexico,Quarter-finals,Belgium,1,True,False,1,True
4,5,1990,Italy,Second round,Republic of Ireland,0,True,False,1,True


In [499]:
df.to_csv('world_cup_penalty_data.csv', index=False) #saves to folder of this notebook if no filepath given.